## Dependencies

In [ ]:
!pip install azureml
!pip install azureml-core

!pip install onnxmltools
!pip install onnxruntime
!pip install tf2onnx

## Get the model

In [ ]:
!wget https://github.com/gfgullo/ImageCaptioning/raw/master/model/caption_model.h5 -P model/
!wget https://github.com/gfgullo/ImageCaptioning/raw/master/model/encode_model.h5 -P model/
!wget https://github.com/gfgullo/ImageCaptioning/raw/master/model/tokenizer.pickle -P model/

## Create the Workspace...

In [1]:
from azureml.core import Workspace

#ws = Workspace.get(name="MachineLearning2", subscription_id='cc0bd342-d77d-41fe-b5be-f1cd80d48167', resource_group='M')


ws = Workspace.create(name='ImageCaptioningWorkspace',
                      subscription_id='cc0bd342-d77d-41fe-b5be-f1cd80d48167',
                      resource_group='M',
                      create_resource_group=True,
                      location='eastus2', exist_ok=True)

Deploying KeyVault with name imagecapkeyvaultd350883b.
Deploying StorageAccount with name imagecapstorage930027103.
Deploying AppInsights with name imagecapinsights4fcdb66c.
Deployed AppInsights with name imagecapinsights4fcdb66c. Took 7.31 seconds.
Deployed KeyVault with name imagecapkeyvaultd350883b. Took 27.69 seconds.
Deployed StorageAccount with name imagecapstorage930027103. Took 31.65 seconds.
Deploying Workspace with name ImageCaptioningWorkspace.
Deployed Workspace with name ImageCaptioningWorkspace. Took 58.62 seconds.


## or load it

## Register the Model

In [4]:
from azureml.core.model import Model

model = Model.register(model_path = "./model",
                       model_name = "ImageCaptioningModel",
                       description = "An Image Captioning model",
                       workspace = ws)

Registering model ImageCaptioningModel


## Define the Estimator

In [28]:
%%writefile caption.py

import pickle
import json
import sys

from azureml.core.model import Model

import numpy as np

from keras.models import load_model
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.image import img_to_array

from PIL import Image
import requests
from io import BytesIO

import traceback


def init():
    
    global caption_model
    global tokenizer
    global encode_model
    global model_path
    
    global MAX_LEN
    global OUTPUT_DIM
    global WIDTH
    global HEIGHT
    
    MAX_LEN = 46
    OUTPUT_DIM = 2048
    WIDTH = 299
    HEIGHT = 299
        
    model_path = Model.get_model_path('ImageCaptioningModel')
    caption_model = load_model(model_path+"/caption_model.h5")
    encode_model = load_model(model_path+"/encode_model.h5")
    
    with open(model_path + '/tokenizer.pickle','rb') as handle:
        tokenizer = pickle.load(handle)



def run(raw_data):
    try:
        
        caption = "startseq"
        
        data = json.loads(raw_data)
        
        if("url" in data):
            url = data["url"]
                
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
        
            img = img.resize((WIDTH, HEIGHT), Image.ANTIALIAS)  
            img = img_to_array(img)
            
        elif("data" in data):
            arr = np.array(data["data"], dtype=np.float32)
            img = arr.reshape((WIDTH,HEIGHT, 3))
        else:
            
            return {"error":"No data provided"}
        
        img = preprocess_input(img)
        img = np.expand_dims(img, axis=0)
        
        x1 = encode_model.predict(img)
        x1 = x1.reshape((1, OUTPUT_DIM))
    
        for i in range(MAX_LEN):
            seq = tokenizer.texts_to_sequences([caption])
            x2 = pad_sequences(seq, maxlen=MAX_LEN)
        
            y = caption_model.predict([x1,x2], verbose=0)
            word = tokenizer.index_word[np.argmax(y)]
        
            if word == "endseq":
                break
      
            caption+=" "+word
    
        caption = caption.replace("startseq","").strip()
        return {"caption":caption}
    
    except Exception as e:
        print(traceback.format_exc())
        return {"error":str(e)}

Overwriting caption.py


## Define the Enviroment

In [29]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_pip_package("numpy")
myenv.add_pip_package("azureml-core")
myenv.add_pip_package("Pillow")
myenv.add_pip_package("keras")
myenv.add_pip_package("tensorflow")
myenv.add_pip_package("scikit-learn")

with open("mlenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

## Make the Container

In [30]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "caption.py",
                                                  runtime = "python",
                                                  conda_file = "mlenv.yml",
                                                  description = "test"
                                                 )

In [31]:
image = ContainerImage.create(name = "image-captioning-container",
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running............................................
Succeeded
Image creation operation finished for image image-captioning-container:4, operation "Succeeded"


## Deploy the Model

In [17]:
from azureml.core.webservice import AciWebservice, Webservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {"data": "image-captioning", "type": "classification"}, 
                                               description = 'An Image Captioning Model')

service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                            image = image,
                                            name = "image-captioning",
                                            workspace = ws)

"""
service = Webservice.deploy(deployment_config = aciconfig,
                                            model_paths="./model",
                                            image_config = image_config,
                                            name = "image-captioning",
                                            workspace = ws)
"""

service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)

Error, there is already a service with name image-captioning found in workspace ImageCaptioningWorkspace



WebserviceException: WebserviceException:
	Message: Error, there is already a service with name image-captioning found in workspace ImageCaptioningWorkspace
	InnerException None
	ErrorResponse {"error": {"message": "Error, there is already a service with name image-captioning found in workspace ImageCaptioningWorkspace"}}

In [27]:
service.get_logs()

'2019-09-01T16:24:14,710416826+00:00 - gunicorn/run \n2019-09-01T16:24:14,716812803+00:00 - iot-server/run \n2019-09-01T16:24:14,731088576+00:00 - nginx/run \n2019-09-01T16:24:14,747645675+00:00 - rsyslog/run \nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2019-09-01T16:24:15,154286578+00:00 - iot-server/finish 1 0\n2019-09-01T16:24:15,206283304+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 19.9.0\nListening at: http://127.0.0.1:31311 (10)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 45\nInitializing logger\nStarting up app insights client\nStarting up request id generator\nStarting up app insight hooks\nInvoking user\'s init function\n2019-09-01 16:24:23,552 | azureml.core.run | DEBUG | Could not load run context RunEnvironmentException:\n\tMessage: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.\n\tInnerExcepti

In [ ]:
service = Webservice(name="image-captioning", workspace = ws)
service.update(image=image)

service.wait_for_deployment(show_output = True)

Running...

## Test the Web Service

### With Link

In [ ]:
import json
import requests

IMG_URL = "https://cdn.pixabay.com/photo/2017/02/20/18/03/cat-2083492_960_720.jpg"

input_data = json.dumps({"url": IMG_URL})

headers = {'Content-Type':'application/json'}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print(json.loads(resp.text))


### With Data

In [ ]:
!wget https://images.financialexpress.com/2018/12/train-18-tracks-660.jpg

In [ ]:
from keras.preprocessing.image import img_to_array
from PIL import Image

import json
import requests


img = Image.open("train-18-tracks-660.jpg")
img = img.resize((299, 299), Image.ANTIALIAS)  
img = img_to_array(img)

input_data = json.dumps({"data": img.tolist()})
headers = {'Content-Type':'application/json'}
resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print(json.loads(resp.text))



## Resources
* https://docs.microsoft.com/bs-latn-ba/azure/machine-learning/service/how-to-deploy-existing-model